In [1]:
import torch
import torch.nn as nn

In [4]:
ones = - torch.ones(3, 3)
print(ones)

tensor([[-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.]])


In [ ]:
class VariationalAutoregressiveNetwork(nn.Module):
    """
    A Variational Autoregressive Network class inheriting from the parent class nn.Module. 
    nn.Module is the generic parent class of models in Pytorch.
    It requires a method called forward. 
    
    Pytorch will be able to recursively recover all parameters of the
    attributes of a nn.Module object provided the attributes have themselves
    type nn.Modules or nn.ModuleList.

    In this implementation we explicitly specify the scale of initialization 
    of the weight matrices.
    """

    def __init__(self,activation=torch.sigmoid, out_activation=None):
        super(VariationalAutoregressiveNetwork, self).__init__()

        self.activation = activation
        self.out_activation = out_activation
        
    def forward(self, x):
        """For the spin variables, we choose sigmoid function ranging
        in $(0,1)$ as an activation function, and the output of the
        network is given by $$\hat s = \sigma(\sum_{j<i}W_{ij}s_j)$$
        """
        x = self.activation(x)
        return x
    
    



For the spin variables, we choose sigmoid function ranging
in $(0,1)$ as an activation function, and the output of the
network is given by $$\hat s = \sigma(\sum_{j<i}W_{ij}s_j)$$

In [ ]:
import torch.nn as nn

class VAN(nn.Module):
    """
    a MLP class inheriting from the parent class nn.Module. 
    nn.Module is the generic parent class of models in Pytorch.
    It requires a method called forward. 
    
    Pytorch will be able to recursively recover all parameters of the
    attributes of a nn.Module object provided the attributes have themselves
    type nn.Modules or nn.ModuleList.

    In this implementation we explicitly specify the scale of initialization 
    of the weight matrices.
    """
    def __init__(self, input_size, activation=torch.sigmoid, out_activation=None, init_scale = 1):
        super(VAN, self).__init__()

        self.input_size = input_size
        self.activation = activation
        self.out_activation = out_activation


        self.W = nn.Linear(input_size,input_size)

        if init_scale is not None:
            for l, layer in enumerate(linears):
                torch.nn.init.normal_(
                    layer.weight, std=init_scale / np.sqrt(layerdims[l])
                )
                torch.nn.init.zeros_(layer.bias)

        self.linears = nn.ModuleList(linears)

    def forward(self, x):
        layers = list(enumerate(self.linears))
        for _, l in layers[:-1]:
            x = self.activation(l(x))
        y = layers[-1][1](x)
        if self.out_activation is not None:
            y = self.out_activation(y)
        return y